In [ ]:
import GMVAE
import torch
import torch.nn as nn
import torch.nn.functional as F
import networks
# Parametri di esempio
batch_size = 1
input_dim = 3 # immagine RGB
z_dim = 8
y_dim = 3
c_dim = 3
img_size = 64
gaussian_size = 108
nz=108
num_classes=2
num_domains=2
x_dim=139968
crop_size=108
use_adain=False
double_layer_ReLUINSConvTranspose=False


# Selezione device (usa GPU se disponibile)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Istanzia encoder e decoder
content_encoder = networks.MD_E_content(input_dim=input_dim, use_cuda=torch.cuda.is_available()).to(device)
attr_encoder = networks.MD_E_attr_concat(input_dim=input_dim,z_dim=gaussian_size, y_dim=num_classes, output_nc=nz, c_dim=num_domains, \
          norm_layer=None, nl_layer=networks.get_non_linearity(layer_type='lrelu'))
decoder = networks.MD_G_multi_concat(input_dim,x_dim,gaussian_size,crop_size, c_dim=num_domains, nz=nz, use_adain=use_adain, double_ConvT=double_layer_ReLUINSConvTranspose)



# Input fittizio (immagini da due domini + condizioni)
x_a = torch.randn(batch_size, input_dim, img_size, img_size, device=device)
x_b = torch.randn(batch_size, input_dim, img_size, img_size, device=device)
c_a = torch.randn(batch_size, c_dim, device=device)
c_b = torch.randn(batch_size, c_dim, device=device)
z_random = torch.randn(batch_size, z_dim, device=device)


# --- Forward content encoder ---
content_a = content_encoder.forward(x_a)
content_b = content_encoder.forward(x_b)
print("Content A:", content_a.shape, "Content B:", content_b.shape)


# --- Forward attribute encoder ---
attr_out_a = attr_encoder.forward(x_a, c_a, temperature=1.0, hard=0)
attr_out_b = attr_encoder.forward(x_b, c_b, temperature=1.0, hard=0)


attr_a = attr_out_a["gaussian"]
attr_b = attr_out_b["gaussian"]
y_a = attr_out_a["categorical"]
y_b = attr_out_b["categorical"]


# Costruzione input come in DRIT++
input_content_forA = torch.cat((content_b, content_a, content_b), 0)
input_content_forB = torch.cat((content_a, content_b, content_a), 0)


input_attr_forA = torch.cat((attr_a, attr_a, z_random), 0)
input_attr_forB = torch.cat((attr_b, attr_b, z_random), 0)


input_y_forA = torch.cat((y_a, y_a, y_a), dim=0)  # shape: (3*B, y_dim)
input_y_forB = torch.cat((y_b, y_b, y_b), dim=0)  # shape: (3*B, y_dim)


input_c_forA = torch.cat((c_a, c_a, c_a), 0)
input_c_forB = torch.cat((c_b, c_b, c_b), 0)


# --- Forward generator (decoder) ---
outA = decoder.forward(input_content_forA, input_attr_forA, input_c_forA, input_y_forA)
outB = decoder.forward(input_content_forB, input_attr_forB, input_c_forB, input_y_forB)


print("Decoder outA keys:", outA.keys())
print("Decoder outB keys:", outB.keys())


# Controlli
assert "x_img" in outA and "x_img" in outB, "Decoder deve produrre immagine ricostruita (x_img)"
assert "x_rec" in outA and "x_rec" in outB, "Decoder deve produrre ricostruzione vettoriale (x_rec)"


assert outA["x_img"].ndim == 4 and outB["x_img"].ndim == 4, "x_img deve essere (B,C,H,W)"
assert outA["x_rec"].ndim == 2 and outB["x_rec"].ndim == 2, "x_rec deve essere (B,features)"


print("\nTutti i test di consistenza passati ✅")

Using device: cpu
Content A: torch.Size([1, 256, 16, 16]) Content B: torch.Size([1, 256, 16, 16])
Attribute Encoder input x_a: torch.Size([1, 3, 64, 64]) c_a: torch.Size([1, 3])


RuntimeError: Given groups=1, weight of size [64, 5, 4, 4], expected input[1, 6, 66, 66] to have 5 channels, but got 6 channels instead